# Investigating Discrepancies in Amount of Frames Extracted

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from data_preprocessing import get_video_frames

In [3]:
#load video clip info dataframe
workspace_path = '/mount/data'
downloads_df = pd.read_csv(workspace_path + '/downloaded_videos.csv')
downloads_df.head()

,video_id,url,title,license,relevant,renamed_title,clip_start,clip_end,duration,frames_collected,frames_reported
0,67OIlq2oMt0,https://youtu.be/67OIlq2oMt0,Hawaiian Islands Humpback Whale National Marin...,creativeCommon,True,video_0000.mp4,10,25,266.0,361.0,473.0
1,EUcMuUBMYJc,https://youtu.be/EUcMuUBMYJc,Tagging expedition: revealing the delicate nur...,creativeCommon,True,video_0001.mp4,44,59,269.0,360.0,441.0
2,dpSuygz7ZmA,https://youtu.be/dpSuygz7ZmA,Your Earth Is Blue: Disentangling a Humpback W...,creativeCommon,True,video_0002.mp4,5,20,102.0,361.0,398.0
3,jfPzlFLEuKk,https://youtu.be/jfPzlFLEuKk,"Humpback Whale || Description, Characteristics...",creativeCommon,True,video_0003.mp4,160,175,238.0,450.0,456.0
4,xGasapMoy4I,https://youtu.be/xGasapMoy4I,Your Earth Is Blue: Humpback Whales,creativeCommon,True,video_0004.mp4,0,15,83.0,451.0,451.0


# Clean up frame directory (testing)

In [63]:
#remove frames from test frame directory
import os
import glob

# files = glob.glob('/workspace/youtube-humpback-whale-classifier/classification/frames/clip_0000_frame*.jpg')

#DELETING FROM WORKSPACE
files = glob.glob(workspace_path + '/frames/clip*.jpg')

for f in files:
    os.remove(f)

# Save frames as .jpeg files in workspace

In [37]:
# #Add new columns to dataframe to keep track of number of frames extracted + reported
# #only run this once
# downloads_df['frames_collected'] = [np.NaN for i in range(0, len(downloads_df))]
# downloads_df['frames_reported'] = [np.NaN for i in range(0, len(downloads_df))]
# downloads_df.head()

In [5]:
for i, row in downloads_df.iterrows():
    clip_title  = row['renamed_title'].replace('_', '_clip_')
    frames_collected, frames_reported = get_video_frames(clip_title, resize=(224, 224))
    downloads_df.at[i, ('frames_collected')] = frames_collected
    downloads_df.at[i, ('frames_reported')] = frames_reported

print('Done extracting all available frames from clips.')
downloads_df.head()

[h264 @ 0x67a1400] mmco: unref short failure
[av1 @ 0x4944cc0] Your platform doesn't suppport hardware accelerated AV1 decoding.
[av1 @ 0x4944cc0] Failed to get pixel format.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0x4944cc0] Missing Sequence Header.
[av1 @ 0

Done extracting all available frames from clips.


,video_id,url,title,license,relevant,renamed_title,clip_start,clip_end,duration,frames_collected,frames_reported
0,67OIlq2oMt0,https://youtu.be/67OIlq2oMt0,Hawaiian Islands Humpback Whale National Marin...,creativeCommon,True,video_0000.mp4,10,25,266.0,361.0,473.0
1,EUcMuUBMYJc,https://youtu.be/EUcMuUBMYJc,Tagging expedition: revealing the delicate nur...,creativeCommon,True,video_0001.mp4,44,59,269.0,360.0,441.0
2,dpSuygz7ZmA,https://youtu.be/dpSuygz7ZmA,Your Earth Is Blue: Disentangling a Humpback W...,creativeCommon,True,video_0002.mp4,5,20,102.0,361.0,398.0
3,jfPzlFLEuKk,https://youtu.be/jfPzlFLEuKk,"Humpback Whale || Description, Characteristics...",creativeCommon,True,video_0003.mp4,160,175,238.0,450.0,456.0
4,xGasapMoy4I,https://youtu.be/xGasapMoy4I,Your Earth Is Blue: Humpback Whales,creativeCommon,True,video_0004.mp4,0,15,83.0,451.0,451.0


# Trying different video reading libraries

https://towardsdatascience.com/lightning-fast-video-reading-in-python-c1438771c4e6 

https://github.com/dmlc/decord

https://medium.com/@haydenfaulkner/extracting-frames-fast-from-a-video-using-opencv-and-python-73b9b7dc9661

In [21]:
from decord import cpu, gpu

downloads_df['frames_reported_decord'] = [np.NaN for i in range(len(downloads_df))]

for i, row in downloads_df.iterrows(): 
    clip = row['renamed_title'].replace('_', '_clip_')
    actual, reported = get_video_frames(clip, max_frames=500, context=cpu(0))
    downloads_df.at[i, ('frames_reported_decord')] = reported

print('Done checking frames')
print(actual, reported)

DECORDError: [12:20:29] /github/workspace/src/video/video_reader.cc:151: Check failed: st_nb >= 0 (-1128613112 vs. 0) ERROR cannot find video stream with wanted index: -1

In [13]:
downloads_df

,video_id,url,title,license,relevant,renamed_title,clip_start,clip_end,duration,frames_collected,frames_reported,frames_reported_decord
0,67OIlq2oMt0,https://youtu.be/67OIlq2oMt0,Hawaiian Islands Humpback Whale National Marin...,creativeCommon,True,video_0000.mp4,10,25,266.0,361.0,473.0,955.0
1,EUcMuUBMYJc,https://youtu.be/EUcMuUBMYJc,Tagging expedition: revealing the delicate nur...,creativeCommon,True,video_0001.mp4,44,59,269.0,360.0,441.0,955.0
2,dpSuygz7ZmA,https://youtu.be/dpSuygz7ZmA,Your Earth Is Blue: Disentangling a Humpback W...,creativeCommon,True,video_0002.mp4,5,20,102.0,361.0,398.0,955.0
3,jfPzlFLEuKk,https://youtu.be/jfPzlFLEuKk,"Humpback Whale || Description, Characteristics...",creativeCommon,True,video_0003.mp4,160,175,238.0,450.0,456.0,955.0
4,xGasapMoy4I,https://youtu.be/xGasapMoy4I,Your Earth Is Blue: Humpback Whales,creativeCommon,True,video_0004.mp4,0,15,83.0,451.0,451.0,955.0
...,...,...,...,...,...,...,...,...,...,...,...,...
363,K1RzTZI9p5U,https://youtu.be/K1RzTZI9p5U,El Secreto del Sueño: Un mito aborigen austral...,creativeCommon,False,video_0425.mp4,117,132,620.0,450.0,598.0,955.0
364,XIhVygewzsM,https://youtu.be/XIhVygewzsM,The World&#39;s Strangest Capital Cities,creativeCommon,False,video_0427.mp4,47,62,475.0,450.0,468.0,955.0
365,nYGZBz5iqe8,https://youtu.be/nYGZBz5iqe8,#HealyHustle | The Elephant Story - What do El...,creativeCommon,False,video_0428.mp4,139,154,189.0,361.0,366.0,955.0
366,uL8pTh8exNI,https://youtu.be/uL8pTh8exNI,하와이 캠핑 빅아일랜드 스펜서 비치파크 (Feat. 마할로의 시크릿 비치도 공개합니다),creativeCommon,False,video_0429.mp4,283,298,664.0,450.0,461.0,955.0


# Save Changes to Downloads DF

In [24]:
downloads_df.drop(downloads_df[downloads_df.frames_collected == 0.0].index, inplace=True)

In [26]:
# update csv with frame counts (actual + reported)
downloads_df.to_csv(workspace_path + '/downloaded_videos.csv', index=False)
downloads_df.to_csv('/workspace/youtube-humpback-whale-classifier/data/downloaded_videos.csv', index=False)